In [3]:
import torch
from torchvision import transforms
import skvideo
skvideo.setFFmpegPath('/usr/local/ffmpeg/bin')
import skvideo.io

from PIL import Image
import numpy as np
from VSFACopy1 import VSFA
from CNNfeatures import get_features
from argparse import ArgumentParser
import time
import os

/data/miniconda3/envs/env-3.6.8/lib/python3.6/site-packages/skvideo/__init__.py:306: UserWarning: ffmpeg/ffprobe not found in path: /usr/local/ffmpeg/bin
  warnings.warn("ffmpeg/ffprobe not found in path: " + str(path), UserWarning)


In [4]:
torch.cuda.is_available(),torch.cuda.device_count()

(True, 8)

In [48]:
os.environ['CUDA_VISIBLE_DEVICE']='1,2,3,4'

In [49]:
a = torch.Tensor(2, 3)

In [50]:
a.get_device()

-1

In [51]:
a = a.cuda()

In [52]:
a.get_device()

0

In [8]:
torch.cuda.set_device(2)

In [9]:
a.get_device()

0

In [10]:
b= torch.Tensor(2, 3).cuda()

In [11]:
b.get_device()

2

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path ="./models/VSFA.pt"
model = VSFA()
model.load_state_dict(torch.load(model_path))  #
model.to(device)

VSFA(
  (ann): ANN(
    (fc0): Linear(in_features=4096, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=128, out_features=128, bias=True)
  )
  (rnn): GRU(128, 32, batch_first=True)
  (q): Linear(in_features=32, out_features=1, bias=True)
)

In [5]:
videos_dir1 = "/cfs/cfs-3cab91f9f/liuzhang/video_data/video_clarity_vid"
video_list1 = os.listdir(videos_dir1)
videos_dir2 = "/cfs/cfs-3cab91f9f/liuzhang/video_data/video_clarity_vid2"
video_list2 = os.listdir(videos_dir2)
videos_dir3 = "/cfs/cfs-3cab91f9f/liuzhang/video_data/video_clarity_vid3"
video_list3 = os.listdir(videos_dir3)
len(video_list1),len(video_list2),len(video_list3)

(7831, 8290, 8681)

In [22]:
feature1 = "/cfs/cfs-3cab91f9f/liuzhang/video_data/CNN_features_mydata/"
feature2 = "/cfs/cfs-3cab91f9f/liuzhang/video_data/CNN_features_mydata2/"

feature1_list = os.listdir(feature1)
feature2_list = os.listdir(feature2)

len(feature1_list),4380-len(feature2_list)

(1537, 953)

In [59]:
import shutil

In [61]:
d = feature1_list[0]
full_path = os.path.join(feature1)

'b3106bj1dms.mp4_resnet-50_res5c--2.npy'

In [ ]:
 shutil.move(full_path, despath)

In [40]:
for f in feature2_list:
    if '_score.npy' in f:
        p= np.load(os.path.join(feature2,f))
        if p ==2:
            print(f)
            break

t14370s7apj.mp4_score.npy


In [31]:
2716/2

1358.0

In [6]:
import pandas as pd

In [7]:
datainfo = "./data/ResultA_C_pic3.csv"
Info = pd.read_csv(datainfo)

video_names1 = list(Info["vid"])

In [8]:
len(video_names1)

2217

In [10]:
video_names=[]
video_names2=[]
video_names3 =[]

In [12]:
for name in video_names1:
    if name+".mp4" in video_list1:
        video_names.append(name)
for name in video_names1:
    if name+".mp4" in video_list2:
        video_names2.append(name)
for name in video_names1:
    if name+".mp4" in video_list3:
        video_names3.append(name)

In [13]:
len(video_names),len(video_names2),len(video_names3)

(0, 0, 2217)

In [23]:
1537+2843

4380

In [33]:
video_names2[446]
name = video_names[1342]
name

'a3219ot59s0'

In [36]:
video_data = skvideo.io.vread(os.path.join(videos_dir1, name+'.mp4'))

AssertionError: Cannot find installation of real FFmpeg (which comes with ffprobe).

In [59]:

a = torch.Tensor(2, 3)
b = torch.Tensor(2, 3)
print(a.get_device())
if torch.cuda.is_available(): # 判断是否支持 CUDA
    a.is_cuda  # False
    a = a.cuda()  # 放到 GPU 上
    
    print(a.get_device())

    

-1
4


In [60]:
torch.cuda.set_device(0)
b = b.cuda()
print(b.get_device())

0


In [28]:
with open("video_info-Copy1.txt","r") as f:
    all_data = f.readlines()

In [29]:
video_dir = "/cfs/cfs-3cab91f9f/liuzhang/video_data/video_clarity_vid"

In [30]:
skvideo.getFFmpegPath()

'/usr/local/ffmpeg/bin'

In [31]:
model_path ="./models/VSFA.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
video_list = os.listdir(video_dir)
model = VSFA()
model.load_state_dict(torch.load(model_path))  #
model.to(device)
model.eval()
A = {}
C = {}

In [32]:
video_list = os.listdir(video_dir)

In [33]:
for data in all_data:
    name = data.split(" ")[0][:-3]+".mp4"
    label = data.split(" ")[0][-2:-1]
    if name not in video_list:
        continue
    video_path = os.path.join(video_dir,name)
    video_data = skvideo.io.vread(video_path)
    
    video_length = video_data.shape[0]
    video_channel = video_data.shape[3]
    video_height = video_data.shape[1]
    video_width = video_data.shape[2]
    print(video_data.shape)
    
    transformed_video = torch.zeros([video_length, video_channel, video_height, video_width])
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    for frame_idx in range(video_length):
        frame = video_data[frame_idx]
        frame = Image.fromarray(frame)
        frame = transform(frame)
        transformed_video[frame_idx] = frame
    print('Video length: {}'.format(transformed_video.shape[0]))
    
    features = get_features(transformed_video, frame_batch_size=32, device=device)
    features = torch.unsqueeze(features, 0)  # batch size 1

        # quality prediction using VSFA

    with torch.no_grad():
        input_length = features.shape[1] * torch.ones(1, 1)
        outputs = model(features, input_length)
        y_pred = outputs[0][0].to('cpu').numpy()
        print("Predicted quality: {}".format(y_pred))
        print("labeled qulity:{}".format(label))
        if label =="A":
            A[name] = y_pred
        elif label =="C":
            C[name] = y_pred
    
    
        
        
 
    
    

(265, 864, 486, 3)
Video length: 265


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /usr/local/app/.cache/torch/checkpoints/resnet50-19c8e357.pth


URLError: <urlopen error [Errno 101] Network is unreachable>